In [2]:
from pyspark import RDD, SparkContext
from collections import defaultdict
from itertools import combinations
import math
import json
import time
from copy import deepcopy
import csv

In [53]:
case_number = 1
filter_threshold = 20
support = 15
input = "ta_feng_all_months_merged.csv"
output = "output.txt"

preprocessed_file = 'customer_product.csv'

test_partitions = 2
sc = SparkContext("local", "HW2").getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=HW2, master=local) created by __init__ at <ipython-input-3-248cfce18ceb>:8 

In [57]:

to_csv_RDD = sc.textFile(input, test_partitions).map(lambda line: line.split(','))
to_csv_RDD.getNumPartitions()
header = to_csv_RDD.first()
to_csv_RDD = to_csv_RDD.filter(lambda data: data != header)

to_csv_RDD = to_csv_RDD.map(lambda x: [str(x[0][1:-1] + "-" + str(int(x[1][1:-1]))), int(x[5][1:-1])])

to_csv_RDD = to_csv_RDD.map(lambda x: ("\"" +str(x[0])+"\"" + "," + str(x[1]) + "\n")).collect()

file = open(preprocessed_file, 'w+')

# writing the data into the file
with file:
    file.write("\"DATE-CUSTOMER_ID\",\"PRODUCT_ID\"\n")
    for row in to_csv_RDD:
        file.write(row)

In [54]:
user_biz_RDD = sc.textFile(preprocessed_file, test_partitions).map(lambda line: line.split(','))
user_biz_RDD.getNumPartitions()
header = user_biz_RDD.first()
user_biz_RDD = user_biz_RDD.filter(lambda data: data != header)
user_biz_RDD = user_biz_RDD.map(lambda x: (x[0], x[1])).groupByKey().map(lambda x: set(x[1])).filter(
    lambda x: len(x) > filter_threshold)


In [56]:
user_biz_RDD.collect()

[{'4710011405133',
  '4710011406123',
  '4710011409056',
  '4710011432832',
  '4710047502011',
  '4710047512515',
  '4710085120628',
  '4710085172696',
  '4710098166811',
  '4710126031890',
  '4710126092105',
  '4710126092129',
  '4710199010778',
  '4710304111147',
  '4710857000011',
  '4711524000433',
  '4711524000891',
  '4711524000907',
  '4711524001041',
  '4713608210912',
  '4714981010038',
  '4719862260427',
  '4901201101021',
  '4903065141502',
  '7610700600863',
  '89782012029',
  '9310034370095',
  '9310135026372'},
 {'4710008242154',
  '4710015103479',
  '4710036006223',
  '4710043105124',
  '4710085120093',
  '4710094020568',
  '4710105045320',
  '4710105045450',
  '4710160013067',
  '4710160013074',
  '4710199010174',
  '4710321871055',
  '4710363522007',
  '4710386123229',
  '4710632001622',
  '4710731041802',
  '4710883000061',
  '4710883000092',
  '4710943101059',
  '4711186319911',
  '4712277820637',
  '4712814168888',
  '4713428000151'},
 {'4710022201496',
  '471003622

In [9]:
total_baskets = user_biz_RDD.count()
total_baskets

25

## Phase 1

In [10]:
subset = user_biz_RDD.collect()

In [11]:
#OG
def pass1(subset, total_baskets, support):
    candidates = []

    baskets = list(subset)
    chunk_size = math.ceil(len(baskets)/int(total_baskets))
    ps = chunk_size * support
    #print(ps)

    # Single Candidates

    singletons = defaultdict(int)
    for b in baskets:
        for item in b:
            singletons[item]+=1

    #print(singletons)
    for k,v in singletons.items():
        if v>=ps:
            candidates.append(frozenset({k}))

    kminusone_cands = candidates
    group_size = 2
    while True:
        group_count = []
        for pair in combinations(kminusone_cands,group_size):
            pair = frozenset().union(*pair)
            pair_count = 0
            for b in baskets:
                if pair.issubset(b):
                    pair_count+=1
            if pair_count>=ps:
                group_count.append(pair)
                #print(pair, pair_count)
        if len(group_count)==0: break;
        #print(group_count)
        if len(group_count)>0:
            next_set = defaultdict(int)
            for k_cand in group_count:
                candidates.append(k_cand)
                for x in k_cand:
                    next_set[x]+=1

        #print(next_set)

        kminusone_cands = []
        for cand,cand_count in next_set.items():
            kminusone_cands.append({cand})
        #print("CANDDIATES of size ", group_size)
        #print("Ewqeqweqwe" ,kminusone_cands)

        group_size+=1

    return(candidates)

pass1(subset, total_baskets, support)

[frozenset({'46'}), frozenset({'33'}), frozenset({'40'}), frozenset({'39'})]

In [12]:
def pass1(subset, total_baskets, support):
    candidates = []
    kminusone_cands = []

    baskets = list(subset)
    chunk_size = float(float(len(subset))/float(total_baskets))
    ps = math.floor(float(chunk_size * support))
    print(ps)

    # Single Candidates

    singletons = defaultdict(int)
    for b in baskets:
        for item in b:
            singletons[item]+=1

    #singletons = sorted(singletons)
    #print(singletons)
    for k,v in singletons.items():
        if int(v)>=int(ps):
            candidates.append({k})
            kminusone_cands.append([k])

    kminusone_cands = sorted(kminusone_cands, key= lambda x: (len(x),x))
    group_size = 2
    intermediate_groups = combinations(kminusone_cands,group_size)
    new_groups = []
    for pair in intermediate_groups:
        new_groups.append(set().union(*pair))
    print(new_groups)

    group_size = 3
    while len(new_groups)>0:
    #for i in range(2):

        kplusonetest = []
        for k_cand in new_groups:
            candidates.append(k_cand)
        for pair in new_groups:
            #pair = set().union(*pair)
            pair_count = 0
            for b in baskets:
                if pair.issubset(b):
                    pair_count+=1
            if pair_count>ps:
                print(pair, pair_count)
                candidates.append(pair)
                kplusonetest.append(pair)
        if len(kplusonetest)==0: break;

        """for k_cand in kplusonetest:
            candidates.append(k_cand)
            print(k_cand)"""
        #group_size+=1
        new_groups = []


        for i in range(len(kplusonetest)):

            for j in range(i+1, len(kplusonetest)):
                #print(list(kplusonetest[i])[:group_size-1])
                #print(list(kplusonetest[j])[:group_size-1])
                if sorted(list(kplusonetest[i])[:group_size-2]) == sorted(list(kplusonetest[j])[:group_size-2]):

                    new_groups.append(kplusonetest[i].union(kplusonetest[j]))
        group_size+=1
        print(new_groups)



        """
        new_candidates = []

        if len(group_count)>0:
            next_set = defaultdict(int)
            for k_cand in group_count:
                candidates.append(k_cand)
                new_candidates.append(k_cand)
                for x in k_cand:
                    next_set[x]+=1

        print(new_candidates)

        kminusone_cands = []
        for cand,cand_count in next_set.items():
            kminusone_cands.append({cand})
        #print("CANDDIATES of size ", group_size)
        #print("Ewqeqweqwe" ,kminusone_cands)

        group_size+=1
        """
    return(candidates)

pass1(subset, total_baskets, support)

15
[{'39', '33'}, {'40', '33'}, {'46', '33'}, {'39', '40'}, {'39', '46'}, {'40', '46'}]


[{'46'},
 {'33'},
 {'40'},
 {'39'},
 {'33', '39'},
 {'33', '40'},
 {'33', '46'},
 {'39', '40'},
 {'39', '46'},
 {'40', '46'}]

In [13]:
def pass1(subset, total_baskets, cands, support):

    candidates = []

    baskets = list(subset)
    chunk_size = math.floor(len(baskets)/int(total_baskets))
    ps = chunk_size * support
    #print(ps)

    # Single Candidates

    singletons = defaultdict(int)
    for b in baskets:
        for item in b:
            singletons[item]+=1

    #print(singletons)
    for k,v in singletons.items():
        if v>=ps:
            candidates.append(frozenset({k}))
    new_singletons = candidates
    kminusone_cands = candidates
    group_size = 2
    while True:
        group_count = []

        for pair in combinations(new_singletons,group_size):
            print("groupsize: ", group_size)
            pair = frozenset().union(*pair)
            pair_count = 0
            for b in baskets:
                if pair.issubset(b):
                    pair_count+=1
            #print(pair_count)
            if pair_count>=ps:
                group_count.append(pair)
                #print(pair, pair_count)
        if len(group_count)==0: break;
        #print(group_count)
        if len(group_count)>0:
            next_set = defaultdict(int)
            for k_cand in group_count:
                candidates.append(k_cand)
                for x in k_cand:
                    next_set[x]+=1

        #print(next_set)

        kminusone_cands = []
        for cand,cand_count in next_set.items():
            kminusone_cands.append({cand})
        #print("CANDDIATES of size ", group_size)
        #print("Ewqeqweqwe" ,kminusone_cands)

        group_size+=1




    return(candidates)

pass1(subset, total_baskets, support)

TypeError: pass1() missing 1 required positional argument: 'support'

In [14]:
#WORKINGGGGG
def pass1(subset, cands, total_baskets, support):

    test_candidates = deepcopy(cands)
    subset = list(subset)
    chunk_size = float(float(len(subset))/float(total_baskets))
    ps = math.floor(float(chunk_size * support))
    #print(ps)
    #print(cands)
    pass1candidates = []
    k = 2

    while len(cands) > 0:
    #for i in range(2):
        k_freq = []
        for candidate in test_candidates:
            cand_support = 0
            for basket in subset:
                if all(element in basket for element in candidate):
                    cand_support += 1

            if cand_support >= ps:
                k_freq.append(candidate)
                pass1candidates.append((tuple(candidate),1))

        if len(k_freq)==0:  break
        #pass1candidates.append(tuple(k_freq))


        kplusonecands = []
        for i in range(len(k_freq)):
            for j in k_freq[i+1:]:
                if k_freq[i][:k-2] == j[:k-2]:
                    x = sorted(list(set().union(k_freq[i],j)))
                    kplusonecands.append(x)
        print(kplusonecands)

        test_candidates = kplusonecands
        k+=1
        #print(kplusonecands)

    print(pass1candidates)
    return pass1candidates


x = pass1(subset, singletons, total_baskets, support)
x

NameError: name 'singletons' is not defined

In [15]:
singletons = user_biz_RDD.flatMap(lambda x: x).sortBy(lambda x: x).distinct().map(lambda x: [x]).sortBy(lambda x:x).collect()
singletons

[['1'],
 ['10'],
 ['11'],
 ['12'],
 ['13'],
 ['14'],
 ['15'],
 ['16'],
 ['17'],
 ['18'],
 ['19'],
 ['2'],
 ['20'],
 ['21'],
 ['22'],
 ['23'],
 ['24'],
 ['25'],
 ['26'],
 ['27'],
 ['28'],
 ['29'],
 ['3'],
 ['30'],
 ['31'],
 ['32'],
 ['33'],
 ['34'],
 ['35'],
 ['36'],
 ['37'],
 ['38'],
 ['39'],
 ['4'],
 ['40'],
 ['41'],
 ['42'],
 ['43'],
 ['44'],
 ['45'],
 ['46'],
 ['47'],
 ['48'],
 ['49'],
 ['5'],
 ['6'],
 ['7'],
 ['8'],
 ['9']]

In [16]:
candidates = user_biz_RDD.mapPartitions(lambda partition: pass1(partition, singletons, total_baskets, support)).reduceByKey(lambda x,y:min(x,y)).map(lambda x: frozenset((x[0]))).sortBy(lambda x: (len(x),x)).collect()
candidates

[frozenset({'23'}),
 frozenset({'25'}),
 frozenset({'3'}),
 frozenset({'38'}),
 frozenset({'39'}),
 frozenset({'46'}),
 frozenset({'11'}),
 frozenset({'15'}),
 frozenset({'28'}),
 frozenset({'35'}),
 frozenset({'41'}),
 frozenset({'42'}),
 frozenset({'43'}),
 frozenset({'10'}),
 frozenset({'24'}),
 frozenset({'26'}),
 frozenset({'33'}),
 frozenset({'34'}),
 frozenset({'40'}),
 frozenset({'44'}),
 frozenset({'16'}),
 frozenset({'17'}),
 frozenset({'9'}),
 frozenset({'33', '34'}),
 frozenset({'34', '44'}),
 frozenset({'11', '15'}),
 frozenset({'11', '35'}),
 frozenset({'11', '41'}),
 frozenset({'11', '43'}),
 frozenset({'15', '35'}),
 frozenset({'17', '9'}),
 frozenset({'41', '43'}),
 frozenset({'42', '46'}),
 frozenset({'33', '39'}),
 frozenset({'11', '17'}),
 frozenset({'11', '9'}),
 frozenset({'11', '15', '35'}),
 frozenset({'11', '17', '9'}),
 frozenset({'11', '41', '43'})]

## Phase 2

In [17]:
def pass2(dataset, candidates):
    itemset_count = defaultdict(int)
    for basket in dataset:
        for candidate in candidates:
            if candidate.issubset(basket):
                itemset_count[candidate]+=1
    #print(candidate, itemset_count)

    final_item_set = []
    for key,value in itemset_count.items():
        final_item_set.append((key,value))

    return final_item_set

pass2(subset, candidates)

[(frozenset({'23'}), 13),
 (frozenset({'25'}), 13),
 (frozenset({'3'}), 12),
 (frozenset({'38'}), 14),
 (frozenset({'46'}), 15),
 (frozenset({'15'}), 10),
 (frozenset({'28'}), 13),
 (frozenset({'42'}), 14),
 (frozenset({'43'}), 11),
 (frozenset({'10'}), 14),
 (frozenset({'33'}), 16),
 (frozenset({'34'}), 14),
 (frozenset({'44'}), 13),
 (frozenset({'16'}), 13),
 (frozenset({'33', '34'}), 10),
 (frozenset({'34', '44'}), 11),
 (frozenset({'42', '46'}), 10),
 (frozenset({'39'}), 15),
 (frozenset({'40'}), 15),
 (frozenset({'17'}), 9),
 (frozenset({'9'}), 12),
 (frozenset({'17', '9'}), 8),
 (frozenset({'33', '39'}), 10),
 (frozenset({'41'}), 11),
 (frozenset({'24'}), 14),
 (frozenset({'26'}), 13),
 (frozenset({'35'}), 9),
 (frozenset({'15', '35'}), 8),
 (frozenset({'41', '43'}), 7),
 (frozenset({'11'}), 12),
 (frozenset({'11', '41'}), 8),
 (frozenset({'11', '15'}), 7),
 (frozenset({'11', '35'}), 8),
 (frozenset({'11', '9'}), 9),
 (frozenset({'11', '15', '35'}), 7),
 (frozenset({'11', '43'}),

In [18]:
freq_item_set = user_biz_RDD.mapPartitions(lambda partition: pass2(partition, candidates)).reduceByKey(lambda x,y: x+y).filter(lambda x: x[1]>=support).map(lambda x: x[0]).sortBy(lambda x:len(x)).collect()
    #groupByKey().map(lambda x:x[0]).collect()
freq_item_set

[frozenset({'33'}), frozenset({'39'}), frozenset({'40'}), frozenset({'46'})]

In [19]:
def write_output(candidates, freq_item_set):
    cand_list = []
    for x in candidates:
        cand_list.append(list(x))

    cand_list = sorted(cand_list, key= lambda x: (len(x),x))
    print(cand_list)

    sorted_item_set = []
    for x in freq_item_set:
        sorted_item_set.append(list(x))


    sorted_item_set = sorted(sorted_item_set, key= lambda x: (len(x),x))


    with open(output, 'w') as f:
        f.write("Candidatesx:")

        cur_len = 0
        for cand in cand_list:
            print(cand)
            l = len(cand)
            if l>cur_len:
                if l == 1:
                    towrite = "(\'" + str(cand[0])+"\')"
                    f.write("\n")
                else:
                    towrite = str(cand)
                    f.write("\n\n")
                f.write(towrite)
                cur_len=l
            else:
                if l == 1:
                    towrite = ",(\'" + str(cand[0])+"\')"
                    f.write(towrite)
                else:
                    cur_len=l
                    towrite = "," + str(cand)
                    f.write(towrite)





        f.write("\n\nFrequent Itemsets:")

        cur_len = 0
        for frq_is in sorted_item_set:
            frq_is = tuple(sorted(frq_is))
            l = len(frq_is)
            if l>cur_len:
                if l == 1:
                    towrite = "(\'" + str(frq_is[0])+"\')"
                    f.write("\n")
                else:
                    towrite = str(frq_is)
                    f.write("\n\n")
                f.write(towrite)
                cur_len=l
            else:
                if l == 1:
                    towrite = ",(\'" + str(frq_is[0])+"\')"
                    f.write(towrite)
                else:
                    cur_len=l
                    towrite = "," + str(frq_is)
                    f.write(towrite)


write_output(candidates, freq_item_set)

[['10'], ['11'], ['15'], ['16'], ['17'], ['23'], ['24'], ['25'], ['26'], ['28'], ['3'], ['33'], ['34'], ['35'], ['38'], ['39'], ['40'], ['41'], ['42'], ['43'], ['44'], ['46'], ['9'], ['11', '15'], ['11', '35'], ['11', '41'], ['11', '43'], ['11', '9'], ['17', '11'], ['17', '9'], ['34', '33'], ['34', '44'], ['35', '15'], ['39', '33'], ['41', '43'], ['42', '46'], ['11', '35', '15'], ['11', '41', '43'], ['17', '9', '11']]
['10']
['11']
['15']
['16']
['17']
['23']
['24']
['25']
['26']
['28']
['3']
['33']
['34']
['35']
['38']
['39']
['40']
['41']
['42']
['43']
['44']
['46']
['9']
['11', '15']
['11', '35']
['11', '41']
['11', '43']
['11', '9']
['17', '11']
['17', '9']
['34', '33']
['34', '44']
['35', '15']
['39', '33']
['41', '43']
['42', '46']
['11', '35', '15']
['11', '41', '43']
['17', '9', '11']


In [41]:
def write_output(candidates, freq_item_set):
    with open(output, 'w') as f:
        f.write("Candidates:\n")
        cand_list = defaultdict(list)
        for x in candidates:
            cand_list[len(x)].append((sorted(tuple(x))))

        for k, val in sorted(cand_list.items()):
            #print(k, sorted(val))
            init = True
            strn = ""
            if k==1:
                for k in sorted(val):
                    if init:
                        strn = strn + "(\'" + str(k[0])+"\')"
                        init = False
                    else:
                        strn = strn + "," + "(\'" + str(k[0])+"\')"
            else:
                for k in sorted(val):
                    if init:
                        strn = strn + str(tuple(k))
                        init = False
                    else:
                        strn = strn + "," + str(tuple(k))
            strn = strn + "\n\n"
            f.write(strn)


        f.write("\nFrequent Itemsets:\n")
        f_ilist = defaultdict(list)
        for x in freq_item_set:
            f_ilist[len(x)].append((sorted(tuple(x))))

        for k, val in sorted(f_ilist.items()):
            #print(k, sorted(val))
            init = True
            strn = ""
            if k==1:
                for k in sorted(val):
                    if init:
                        strn = strn + "(\'" + str(k[0])+"\')"
                        init = False
                    else:
                        strn = strn + "," + "(\'" + str(k[0])+"\')"
            else:
                for k in sorted(val):
                    if init:
                        strn = strn + str(tuple(k))
                        init = False
                    else:
                        strn = strn + "," + str(tuple(k))
            strn = strn + "\n\n"
            f.write(strn)

write_output(candidates, freq_item_set)